In [2]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [4]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [6]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [8]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [10]:
# !fc-cache -f

In [11]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [12]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [19]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [20]:
%%capture pwd
!pwd

In [21]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('oecd-healthcare', 300, 500)

In [22]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [23]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [24]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                            if c in i['encoding'][x]['axis']:
                                i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [25]:
df = pd.read_excel("raw/oech.xlsx")
df.columns=['country','total','gov','vol']
for c in ['total','gov','vol']:
    df[c]=df[c].astype(str).str.replace(' ','').str.replace('\xa0','').astype(float)
df['country']=df['country'].str.replace("(People's Republic of)",'')
df=df.sort_values(by='total')

In [26]:
readme, f, fc = save(df,"fig1_health",LOCAL)

,country,total,gov,vol
22,India,212.0,78.5,132.2
23,Indonesia,405.0,222.7,182.3
37,Peru,759.5,522.8,236.7
8,China,978.8,536.4,442.3
33,Mexico,1181.0,626.2,554.8


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 22 to 49
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  50 non-null     object 
 1   total    50 non-null     float64
 2   gov      50 non-null     float64
 3   vol      50 non-null     float64
dtypes: float64(3), object(1)
memory usage: 2.0+ KB


In [28]:
base = alt.Chart(fc).mark_rule(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "total:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="USD / capita",
            titleY=-25,
            titleX=257,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=11,
            orient="bottom",
            labelAngle=0,
            format='$s'
        ),
    ),
    y=alt.Y(
        "country:N",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelFontSize=11,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

pipe1=base.mark_rule(color=colors['eco-dot']).encode(x='total:Q')
pipe2=base.mark_rule(color=colors['eco-light-blue']).encode(x='gov:Q')
pipe3=base.mark_rule(color=colors['eco-turquiose']).encode(x='vol:Q')
points1=base.mark_circle(color=colors['eco-dot'],size=40).encode(x='total:Q')
points2=base.mark_circle(color=colors['eco-light-blue']).encode(x='gov:Q')
points3=base.mark_circle(color=colors['eco-turquiose']).encode(x='vol:Q')
dx=255
l1=points1.mark_circle(color=colors['eco-dot'],size=40,xOffset=dx+7).encode(x='total:Q').transform_filter('datum.country=="Mexico"')
t1=l1.mark_text(text='Total',color=colors['eco-dot'],yOffset=0,align='right',dx=dx)
l2=l1.mark_circle(color=colors['eco-light-blue'],size=40,yOffset=15,xOffset=dx+7).encode(x='total:Q')
t2=l2.mark_text(text='Government / Compulsory',yOffset=15,color=colors['eco-light-blue'],align='right',dx=dx)
l3=l1.mark_circle(color=colors['eco-turquiose'],size=40,yOffset=30,xOffset=dx+7).encode(x='total:Q')
t3=l3.mark_text(text='Voluntary',color=colors['eco-turquiose'],yOffset=30,align='right',dx=dx)

layer1 = (
    (base+points1+points2+points3+l1+t1+l2+t2+l3+t3).properties(height=600, width=300, 
    title={'text':"Health spending",'fontSize':16,'dx':83,'color':colors["eco-gray"],'subtitleColor':colors["eco-gray"],
           'anchor':'start','subtitle':'source: OECD, 2022 or latest available','subtitlePadding':10,'align':'left'})
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=base.mark_image(width=40,height=40,align='right',baseline='top',yOffset=-610,opacity=mo,xOffset=10).encode(x='total:Q',y=alt.Y('country:N',
    sort=[]),
    url='img:N')\
    .transform_calculate(img='"https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"')\
    .transform_filter('datum.country=="United States"')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_health



alt.LayerChart(...)

alt.LayerChart(...)

# Post-process

## Clean up `SVG`s
`base64 URI` encode images

In [29]:
from base64 import b64encode

In [30]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [31]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

fig1_health.svg
fig1_health_dark.svg
fig1_health_local.svg
fig1_health_local_dark.svg


## README

In [32]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_health.csv)  
GitHub: [fig1_health](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/oecd-healthcare)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_health_local.png) [`svg`](visualisation/fig1_health_local.svg) [`json`](visualisation/fig1_health_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_health.png) [`svg`](visualisation/fig1_health.svg) [`json`](visualisation/fig1_health.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_health_local_no_branding.png) [`svg`](visualisation/fig1_health_local_no_branding.svg) [`json`](visualisation/fig1_health_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_health_no_branding.png) [`svg`](visualisation/fig1_health_no_branding.svg) [`json`](visualisation/fig1_health_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_health_local_dark.png) [`svg`](visualisation/fig1_health_local_dark.svg) [`json`](visualisation/fig1_health_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_health_dark.png) [`svg`](visualisation/fig1_health_dark.svg) [`json`](visualisation/fig1_health_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_health_local_no_branding_dark.png) [`svg`](visualisation/fig1_health_local_no_branding_dark.svg) [`json`](visualisation/fig1_health_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_health_no_branding_dark.png) [`svg`](visualisation/fig1_health_no_branding_dark.svg) [`json`](visualisation/fig1_health_no_branding_dark.json)   

!["fig1_health"](visualisation/fig1_health.svg "fig1_health")

  

!["fig1_health_dark"](visualisation/fig1_health_dark.svg "fig1_health")

